<a href="https://colab.research.google.com/github/FernandaSzAdr/KNN_cromossome/blob/master/TesteRede3_Binaria_Base2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/




---



In [2]:
import os
import csv
!pip install imutils
import imutils
from skimage import transform
import numpy as np
import keras
from keras.regularizers import l2
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, concatenate, Input, Conv2D
from keras.layers import MaxPooling2D, GlobalMaxPooling2D
from keras.optimizers import SGD
from keras.layers.core import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import itertools
import matplotlib.pyplot as plt

Using TensorFlow backend.




---



## Obtenção de dados estruturais

**Links interessantes:**

*   https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contours_more_functions/py_contours_more_functions.html
*   https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_properties/py_contour_properties.html
*   https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html
*   https://docs.python.org/3/library/pickle.html
*   https://stackoverflow.com/questions/48456900/how-can-i-pickle-a-python-object-into-a-csv-file





In [0]:
def reformatar_vetor(images):
  saida = []
  for image in images:
    arquivo = image.split('/')[1]
    saida.append(arquivo)
    
  return saida

In [0]:
def normalizar(vetor):
  saida = []
  for item in vetor:
    valor = (item-min(vetor))/(max(vetor)-min(vetor))
    
    saida.append(valor)
  
  return saida
  

In [0]:
def concatenar_vetores(areas, perimetros, aspect_ratios, extents, solidities):
  informacoes = []
  for i in range(len(areas)):
    informacao = []
    informacao.append(areas[i])
    informacao.append(perimetros[i])
    informacao.append(aspect_ratios[i])
    informacao.append(extents[i])
    informacao.append(solidities[i])
    
    informacoes.append(informacao)
  
  return informacoes

In [0]:
def pegar_informacoes_recortes(train_arq):  
  f = open('drive/My Drive/Base_CNN/Binarizado2/posicoes_segmentada.csv', 'r')
  
  areas = []
  perimetros = []
  aspect_ratios = []
  extents = []
  solidities = []
  
  try:
    leitor = csv.reader(f)
    leitor = list(leitor)
    leitor.pop(0)

    for linha in leitor:
      if len(linha) > 0:
        arquivo = linha[2]
        arquivo = arquivo + '.tif'
        
        if arquivo in train_arq:
          area = float(linha[7])
          areas.append(area)
          
          perimetro = float(linha[8])
          perimetros.append(perimetro)
          
          aspect_ratio = float(linha[9])
          aspect_ratios.append(aspect_ratio)
          
          extent = float(linha[10])
          extents.append(extent)
          
          solidity = float(linha[11])
          solidities.append(solidity)
          
  finally:
    f.close()
  
  areas = normalizar(areas)
  perimetros = normalizar(perimetros)
  aspect_ratios = normalizar(aspect_ratios)
  extents = normalizar(extents)
  solidities = normalizar(solidities)
 
  return concatenar_vetores(areas, perimetros, aspect_ratios, extents, solidities)

## Funções

In [0]:
def converte(regiao):
   regiao = np.array(regiao).astype('float32')/255
   regiao = transform.resize(regiao, (120, 120, 3))
   regiao = np.expand_dims(regiao, axis=0)
   return regiao


In [0]:
def ler_images_teste(arquiteture):
  y_true = []
  y_predict = []
  images_test = os.listdir(test_path)
  local = 'drive/My Drive/Base_CNN/Binarizado2/Resultado_Image_Rede3/' + arquiteture
  if os.path.isdir(local) == False:
    os.mkdir(local)
  
  for img in images_test:
    # Pega cada imagem separadamente do folder de teste
    image = cv2.imread(test_path + '/' + img)
    image = imutils.resize(image, width=1000)
    f = open('drive/My Drive/Base_CNN/Binarizado2/posicoes_segmentada.csv', 'r')
    
    copia = image.copy()

    try:
      leitor = csv.reader(f)
      leitor = list(leitor)
      leitor.pop(0)
      for linha in leitor:
        if len(linha) > 0:
          # Para cada região dessa imagem:
          if linha[0] == img:
            posicoes = []
            x = int(linha[3])
            y = int(linha[4])
            w = int(linha[5])
            h = int(linha[6])
            
            # Label da região
            label = linha[1]
            if label == 'NC':
              y_true.append(0)
            else:
              y_true.append(1)
            
            # Região
            regiao = image[y: y + h, x: x + w]
            regiao = converte(regiao)
            
            # Coloca a região na função e obtem o resultado
            predicao = model.predict(regiao,batch_size=bSize,verbose=0)
            print(predicao)
            
            # Se é um cromossomo:
            if predicao[0][0] > 0.5:
              y_predict.append(1)
              cv2.rectangle(copia,(x, y),(x+w, y+h),(0,255,0), 1)
            # Se não for um cromossomo:
            else:
              y_predict.append(0)
              cv2.rectangle(copia,(x, y),(x+w, y+h),(0,0,255), 1)
                        
    finally:
      f.close()
    
    img_nova = img.split('.tif')[0] + '.png'
    cv2.imwrite(local + '/' + img_nova, copia)
  
  return y_true, y_predict




---



## Modelos

In [0]:
def model_1():
  input_image = Input(shape=(120, 120, 3))
  input_dados = Input(shape=(5,))  # len(dados_recortes[0]) 
  
  x = Conv2D(64, (3, 3))(input_image)
  x = Activation('relu')(x)
  x = Conv2D(64, (3, 3))(x)
  x = Activation('relu')(x)
  x = Dropout(0.2)(x)
  x = MaxPooling2D((2, 2))(x)
  
  x = Conv2D(32, (3, 3))(x)
  x = Activation('relu')(x)
  x = Conv2D(32, (3, 3))(x)
  x = Activation('relu')(x)
  x = Dropout(0.2)(x)
  x = MaxPooling2D((2, 2))(x)
  
  x = Conv2D(16, (3, 3))(x)
  x = Activation('relu')(x)
  x = Conv2D(16, (3, 3))(x)
  x = Activation('relu')(x)
  x = Dropout(0.2)(x)
  x = MaxPooling2D((2, 2))(x)
  
  x = Conv2D(8, (3, 3))(x)
  x = Activation('relu')(x)
  x = Conv2D(8, (3, 3))(x)
  x = Activation('relu')(x)
  x = Dropout(0.2)(x)
  x = MaxPooling2D((2, 2))(x)
  
  x = Flatten()(x)
  x = concatenate([x, input_dados])
  
  x = Dense(1024)(x)
  x = Activation('sigmoid')(x)
  x = Dense(512)(x)
  x = Activation('sigmoid')(x)
  
  output = Dense(2)(x)
  output = Activation('softmax')(output)
  
  model = Model(inputs=[input_image, input_dados], outputs=[output])
  
  return model


In [0]:
def model_13():
  input_image = Input(shape=(120, 120, 3))
  input_dados = Input(shape=(5,))  # len(dados_recortes[0]) 

  x = Conv2D(32, (3, 3))(input_image)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.25)(x)

  x = Conv2D(64, (5, 5))(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(2, 2)(x)
  x = BatchNormalization()(x)
  x = Dropout(0.25)(x)

  x = Conv2D(128, (7, 7))(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(2, 2)(x)
  x = BatchNormalization()(x)
  x = Dropout(0.25)(x)

  x = Conv2D(128, (1,1))(x)
  x = GlobalMaxPooling2D()(x)

  x = Flatten()(x)
  x = concatenate([x, input_dados])

  x = Dense(256)(x)
  x = Activation('relu')(x)
  x = Dropout(0.5)(x)

  output = Dense(2)(x)
  output = Activation('sigmoid')(output)

  model = Model(inputs=[input_image, input_dados], outputs=[output])
  
  return model

# Implementação

## Variaveis globais



*   Batch size = 32
*   Image size = 120x120



In [0]:
train_path = 'drive/My Drive/Base_CNN/Binarizado3/Train'
test_path = 'drive/My Drive/Base_CNN/Binarizado3/Test'
validation_path = 'drive/My Drive/Base_CNN/Binarizado3/Validation'

In [0]:
results = {}

In [0]:
bSize = 32

In [0]:
datagen = ImageDataGenerator(featurewise_center=True,
                                  featurewise_std_normalization=True)

In [18]:
train_batches = datagen.flow_from_directory(train_path, target_size=(120, 120), 
                                            classes=['NC', 'C'], 
                                            batch_size=bSize)
validation_batches = datagen.flow_from_directory(validation_path, 
                                                 target_size=(120, 120), 
                                                 classes=['NC', 'C'], 
                                                 batch_size=bSize)

Found 4230 images belonging to 2 classes.
Found 1100 images belonging to 2 classes.


In [0]:
train_arq = reformatar_vetor(train_batches.filenames)

In [0]:
valid_arq = reformatar_vetor(validation_batches.filenames)

In [0]:
dados_recortes_train = pegar_informacoes_recortes(train_arq)

In [0]:
dados_recortes_valid = pegar_informacoes_recortes(valid_arq)

In [0]:
stepsPerEpoche = len(train_batches)/bSize
validationSteps = len(validation_batches)/bSize

In [24]:
train_batches.class_indices

{'C': 1, 'NC': 0}



---



## 1 tentativa 


**Descrição**


*   Modelo descrito no artigo "Crowdsourcing for Chromosome Segmentation and Deep Classification"



In [0]:
architecture = 'architecture_1'

### Modelo

In [0]:
model = model_1()

In [0]:
model.compile(SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
              loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
checkpointer = ModelCheckpoint(filepath='drive/My Drive/Resultado2/Segmentado/weights_' + architecture + '.hdf5', verbose=1, save_best_only=True)

In [37]:
len(train_batches), len(dados_recortes_train)

(133, 4230)

In [35]:
dados_recortes_train

[[0.00033390839779620456,
  0.014194769552731962,
  0.20589730554143362,
  0.49134620735711504,
  1.0],
 [0.0008904223941232122,
  0.019213378370900685,
  0.09761057447890187,
  0.6551282764761535,
  1.0],
 [0.00016695419889810228,
  0.011076605615436072,
  0.09761057447890187,
  0.21837609215871778,
  0.75],
 [0.0024486615838388336,
  0.032190428444172484,
  0.09761057447890187,
  0.5882784523459337,
  0.9777777777777777],
 [0.0005565139963270077,
  0.015664685982717693,
  0.13370615149974577,
  0.5459402303967945,
  1.0],
 [0.0, 0.007097384776365981, 0.025419420437214027, 0.0, 0.0],
 [0.0, 0.007097384776365981, 0.025419420437214027, 0.0, 0.0],
 [0.0017251933886137235,
  0.028211207816620943,
  0.09761057447890187,
  0.5641382380766877,
  0.9393939393939394],
 [0.0007791195948578107,
  0.017134602412703422,
  0.09761057447890187,
  0.5732372419166343,
  1.0],
 [0.00022260559853080306,
  0.010646077164548972,
  0.15175394001016773,
  0.43675218431743557,
  1.0],
 [0.001391284990817519,

In [38]:
training = model.fit([train_batches, dados_recortes_train], steps_per_epoch=stepsPerEpoche, 
                     epochs=50, verbose=2, callbacks=[checkpointer])

AttributeError: ignored

In [0]:
model.load_weights('drive/My Drive/Resultado2/Segmentado/weights_' + architecture + '.hdf5')

In [0]:
# plotar loss e accuracy para os datasets 'train' e 'test'
plt.style.use("ggplot")
plt.figure()
plt.plot(training.history['acc'], label='Accuracy')
plt.plot(training.history['loss'], label='Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title("Training Loss and Accuracy")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('drive/My Drive/Resultado2/Segmentado/' + architecture + '.png', bbox_inches='tight')

### Teste

In [0]:
y_true, y_predict = ler_images_teste(architecture)

In [0]:
results[architecture] = precision_score(y_true, y_predict, average='binary')
print('Test Precision: {:.2f}%'
      .format(100 * results[architecture]))

## 13 tentativa



Modelo adaptado do **Lucas Valentim**

*   Utilizado outra função de normalização, diferente da utilizada por ele.

In [0]:
architecture = 'architecture_13'

### Modelo

In [0]:
model = model_13()

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
checkpointer = ModelCheckpoint(filepath='drive/My Drive/Resultado2/Segmentado/weights_' + architecture + '.hdf5', verbose=1, save_best_only=True)

In [0]:
training = model.fit([train_batches, dados_recortes_train], steps_per_epoch=stepsPerEpoche, 
                     epochs=50, verbose=2, callbacks=[checkpointer])

In [0]:
model.load_weights('drive/My Drive/Resultado2/Segmentado/weights_' + architecture + '.hdf5')

In [0]:
# plotar loss e accuracy para os datasets 'train' e 'test'
plt.style.use("ggplot")
plt.figure()
plt.plot(training.history['acc'], label='Accuracy')
plt.plot(training.history['loss'], label='Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title("Training Loss and Accuracy")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('drive/My Drive/Resultado2/Segmentado/' + architecture + '.png', bbox_inches='tight')

### Teste

In [0]:
y_true, y_predict = ler_images_teste(architecture)

In [0]:
results[architecture] = precision_score(y_true, y_predict, average='binary')
print('Test Precision: {:.2f}%'
      .format(100 * results[architecture]))